In [1]:
!pip install langchain
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install azure-storage-blob
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2ef5bd5421626646ecbd7f7c15c862b35c073561b63c318b44ba44d5e160e5be
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.7 MB/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (206 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [1]:
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title
from sentence_transformers import SentenceTransformer,util
import faiss
from io import BytesIO
from typing import Tuple, List
import numpy as np
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores.faiss import FAISS
import torch
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings
device = torch.device('cuda'if torch.cuda.is_available() else'cpu')
print(device)

cuda


In [5]:
account_name = "ragvectordbcontainer"
account_key ="ZWgcn9h9kHpZVCui58mUkq+PfHBb3FT+qtZmIbH1OR8WwC81YtMzmnfr/lheCiXc3Jdt+krjmStM+ASt4o9liQ=="
container_name ="vecctordbcontainer1"
model = "all-MiniLM-L6-v2"

In [ ]:
def getEmbeddings(text,model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name).to(device)
    embeddings = model.encode(text)
    return embeddings

In [2]:
class Document:
  def __init__(self, page_content,embedding,  metadata,id):
      self.page_content = page_content
      self.embedding = embedding
      self.metadata = metadata
      self.id = id

In [3]:
# Function to embed and store chunks in vector database
def embed_and_store_chunks(folder_path):
    # Initialize the SentenceTransformer model
    model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

    # Create a list to store Document objects
    doc_chunks = []
    for filename in os.listdir(folder_path):
      if filename.endswith('pdf'):
        pdf_path = os.path.join(folder_path, filename)
        try:
          elements = partition(pdf_path)
        except Exception as e:
          print(f"An error occurred when trying to partition the file: {e,filename}")
          continue
        chunks = chunk_by_title(elements, new_after_n_chars=1500, combine_text_under_n_chars=700)
        # Embed each chunk and create Document objects
        for i, chunk in enumerate(chunks):
            # Embed the chunk using SentenceTransformer
            chunk_embedding = model.encode(chunk.text)
            page_number = chunk.metadata.page_number
            doc_id = f"{filename}-{i}"
            # Create a Document object for the chunk
            doc = Document(page_content=chunk.text, embedding=chunk_embedding ,metadata={"page": page_number},id=doc_id)
            doc.metadata["source"] = f"{doc.metadata['page']}-{doc_id}"
            doc.metadata["filename"] = filename
            doc_chunks.append(doc)
        print(filename,"complete")
    metadatax = [doc.metadata for doc in doc_chunks]  # Extract metadata from each Document
    idx = [doc.id for doc in doc_chunks]  # Extract ID from each Document
    # Extract just the embeddings from each Document
    embeddings = [model.encode(doc.page_content) for doc in doc_chunks]
    #Stores all encoded embeddings in the vector DB
    vectorstore_faiss = FAISS.from_embeddings([(doc.page_content, model.encode(doc.page_content)) for doc in doc_chunks], model,metadatas=metadatax,ids=idx)
    return vectorstore_faiss

# Embed and store chunks in vector database
vector_db = embed_and_store_chunks("/content/test/")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Bamboo  Bamboo Plantation.pdf complete


In [4]:
def saveVectorDB(vector_db,path):
    vector_db.save_local(path)

def loadVectorDB(path,model='all-MiniLM-L6-v2'):
    vector_db = FAISS.load_local(path,model)
    return vector_db

# saveVectorDB(vector_db,"faiss_index")

In [6]:
def upload_folder_to_blob(account_name, account_key, container_name, local_folder_path, remote_folder_name):
    # Create a connection string
    connect_str = f'DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net'

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Get a container client
    container_client = blob_service_client.get_container_client(container_name)

    # Iterate through local files and upload them to Azure Blob Storage
    for root, dirs, files in os.walk(local_folder_path):
        for file_name in files:
            local_file_path = os.path.join(root, file_name)
            blob_name = os.path.join(remote_folder_name, os.path.relpath(local_file_path, local_folder_path)).replace("\\", "/")

            # Create a BlobClient and upload the file
            blob_client = container_client.get_blob_client(blob_name)
            with open(local_file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)

    print(f"Folder '{local_folder_path}' uploaded to Azure Blob Storage container '{container_name}/{remote_folder_name}'.")


upload_folder_to_blob(account_name, account_key, container_name, "/content/faiss_index", "faiss_index")


Folder '/content/faiss_index' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


In [7]:

def download_folder_from_blob(account_name, account_key, container_name, remote_folder_name, local_folder_path):
 # Create a connection string
 connect_str = f'DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net'

 # Create a BlobServiceClient
 blob_service_client = BlobServiceClient.from_connection_string(connect_str)

 # Get a container client
 container_client = blob_service_client.get_container_client(container_name)

 # List blobs in the specified folder
 blob_list = container_client.list_blobs(name_starts_with=remote_folder_name)

 # Download each blob to the local folder
 for blob in blob_list:
     blob_name = blob.name

     # Extract the base file name from the blob name
     base_file_name = os.path.basename(blob_name)

     local_file_path = os.path.join(local_folder_path, base_file_name)

     # Create necessary directories
     os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

     # Create a BlobClient and download the file
     blob_client = container_client.get_blob_client(blob_name)
     with open(local_file_path, "wb") as data:
         data.write(blob_client.download_blob().readall())

 print(f"Folder '{container_name}/{remote_folder_name}' downloaded to '{local_folder_path}'.")
# download_folder_from_blob(account_name, account_key, container_name, "faiss_index", "/content/faiss_index2")


In [15]:
def addPDFtoVectorDB(filepath,vectorDBpath,model='all-MiniLM-L6-v2' ):
    emb_model = SentenceTransformer(model).to(device)
    doc_chunks = []
    download_folder_from_blob(account_name, account_key, container_name, "faiss_index", vectorDBpath)
    vector_db = FAISS.load_local(vectorDBpath,model)
    if filepath.endswith('pdf'):
      filename = os.path.basename(filepath)
      print(filename,"started")
      try:
        elements = partition(filepath)
        chunks = chunk_by_title(elements, new_after_n_chars=1500, combine_text_under_n_chars=700)
        for i, chunk in enumerate(chunks):
              # Embed the chunk using SentenceTransformer
              chunk_embedding = emb_model.encode(chunk.text)
              page_number = chunk.metadata.page_number
              doc_id = f"{filename}-{i}"
              # Create a Document object for the chunk
              doc = Document(page_content=chunk.text, embedding=chunk_embedding ,metadata={"page": page_number},id=doc_id)
              doc.metadata["source"] = f"{doc.metadata['page']}-{doc_id}"
              doc.metadata["filename"] = filename
              doc_chunks.append(doc)
        print(filename,"complete")
        metadatax = [doc.metadata for doc in doc_chunks]  # Extract metadata from each Document
        idx = [doc.id for doc in doc_chunks]  # Extract ID from each Document
        # Extract just the embeddings from each Document
        embeddings = [emb_model.encode(doc.page_content) for doc in doc_chunks]
        # Add the embeddings to the vectorstore
        vector_db.add_embeddings(list(zip([doc.page_content for doc in doc_chunks], embeddings)), metadatas=metadatax, ids=idx)
        saveVectorDB(vector_db,vectorDBpath)
        upload_folder_to_blob(account_name, account_key, container_name, vectorDBpath, "faiss_index")
      except Exception as e:
        print(f"An error occurred when trying to partition the file: {e,filename}")

# addPDFtoVectorDB("/content/Ethem Alpaydin-Introduction to Machine Learning-The MIT Press (2014).pdf","/content/faiss_index")
folder_path = "/content/"
vectorDBpath ="/content/faiss_index/"
for filename in os.listdir(folder_path):
   if filename.endswith('.pdf'):
       pdf_path = os.path.join(folder_path, filename)
       addPDFtoVectorDB(filepath=pdf_path, vectorDBpath=vectorDBpath)

Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
Lacquered Dreams - Promoting Livelihoods Through Lac.pdf started
Lacquered Dreams - Promoting Livelihoods Through Lac.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
IPM in Brinjal .pdf started
IPM in Brinjal .pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
Weaving Livelihoods - A Socio-Economic Impact Study of Special SGSY Tasar Projects in Bihar and Jharkhand.pdf started
Weaving Livelihoods - A Socio-Economic Impact Study of Special SGSY Tasar Projects in Bihar and Jharkhand.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
SELCO-Foundation_-65-livelihoods-solutions-from-ground.pdf started
SELCO-Foundation_-65-livelihoods-solutions-from-ground.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
Chick Pea - Value Chain analysis.pdf started
Chick Pea - Value Chain analysis.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
FPC registration procedure.pdf started
FPC registration procedure.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
SRI Vidhi Se Dhan Hetu Prashikshan Pustika .pdf started
SRI Vidhi Se Dhan Hetu Prashikshan Pustika .pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
FINGER MILLET - PPT - trg by Ganapathy KN .pdf started
FINGER MILLET - PPT - trg by Ganapathy KN .pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.
Ethem Alpaydin-Introduction to Machine Learning-The MIT Press (2014).pdf started
Ethem Alpaydin-Introduction to Machine Learning-The MIT Press (2014).pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.
Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index/'.


Lac - Value Chain Analysis Report.pdf started
Lac - Value Chain Analysis Report.pdf complete
Folder '/content/faiss_index/' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


In [16]:
vectorDBpath = "/content/faiss_index2"
def semanticSearch(Query,k,vectorDBpath,model='all-MiniLM-L6-v2'):
  download_folder_from_blob(account_name, account_key, container_name, "faiss_index",vectorDBpath )
  vector_db = FAISS.load_local(vectorDBpath,model)
  emb_model = SentenceTransformer(model)
  e2 = emb_model.encode(Query)
  results =vector_db.similarity_search_by_vector(e2,k)
  return results

text = "What is KNN?"
results = semanticSearch(text,5,vectorDBpath)

for result in results:
  print(result.page_content)
  print(result.metadata)

  print("-------------------------------------------------------------------------------")

Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index2'.
k-nn classifier

The k-nn classiﬁer assigns the input to the class having most examples among the k neighbors of the input. All neighbors have equal vote, and the class having the maximum number of voters among the k neighbors is chosen. Ties are broken arbitrarily or a weighted vote is taken. k is generally taken to be an odd number to minimize ties: Confusion is

193

194

8 Nonparametric Methods

2

x
{'page': 217, 'source': '217-Ethem Alpaydin-Introduction to Machine Learning-The MIT Press (2014).pdf-1016', 'filename': 'Ethem Alpaydin-Introduction to Machine Learning-The MIT Press (2014).pdf'}
-------------------------------------------------------------------------------
Voronoi tesselation

generally between two neighboring classes. A special case of k-nn is the nearest neighbor classiﬁer where k = 1 and the input is assigned to the class of the nearest pattern. This divides the space in the form of a